# Script to wrap up Mathematica equations as a C library

19-02-2018 LT
To be run after `concerted_rots_generic.nb`

In [149]:
from datetime import datetime

## Global definitions

In [150]:
Timestamp="//generated by Precompiler.ipynb on "+str(datetime.now())
args_impl_B=["struct cr_input_data bb"]
args_impl_A=["gsl_vector *T","const struct cr_input_data bb"]
args_expl  =["struct expl_data cr_in"]

            
hl_idx=["1","2","3","4","5","6","7"]
ll_idx=["12","13","23","14","24","34","33"]

### Filenames for C source

In [151]:
header_name="CR_precomp.h"
src_name="CR_precomp.c"

In [152]:
eqn_fname="CR_T7.c"
der_fname="CR_DerT7.c"
jac_fname="CR_JacT7.c"

## C Header

In [153]:
def write_prototypes(fout,ret_type, func_name, func_args=[],comment="",idx_list=[]):
    if(comment!=""):
        fout.write(comment+"\n")
    if(len(idx_list)==0):
        str=ret_type+" "+func_name+" "+"("
        for a in func_args[:-1]:
            str+=a+","
        str+=func_args[-1]+");"
        fout.write(str+"\n")
    else:
        for idx in idx_list:
            str=ret_type+" "+func_name+idx+" "+"("
            for a in func_args[:-1]:
                str+=a+","
            str+=func_args[-1]+");"
            fout.write(str+"\n")

In [154]:
HDR_1="""
#ifndef __conc_rot_precomputed
#define __conc_rot_precomputed
#include <gsl/gsl_vector.h>
#include <gsl/gsl_matrix.h>
#include <gsl/gsl_linalg.h>
#include "../lib/messages.h"
#define EPSIL_DET (0.0000001)

typedef struct cr_input_data{
    gsl_vector *dihed_angles;
    gsl_vector *bend_angles;
    gsl_vector *r;
    gsl_vector *d;
} cr_input_data;

void alloc_cr_input_data(cr_input_data *bb);
void free_cr_input_data(cr_input_data *bb);
void memcpy_cr_input_data(cr_input_data *bb_a, cr_input_data *bb_b);

struct expl_data{
    double c [7]; //phi and psi angles
    double s [7];
    double ca[7]; //alpha angle
    double sa[7];
    double d [7]; //displacement orthogonal to z
    double r [7]; //displacement along z
};
"""
FTR="#endif //__conc_rot_precomputed"

In [155]:
with open(header_name,'w') as hdr_file:
    hdr_file.write(Timestamp+HDR_1)
    #--high level functions
    #----implicit arguments
    write_prototypes(hdr_file,"void","T7",args_impl_A)
    write_prototypes(hdr_file,"int","TmT7_t",args_impl_A,comment="//Tangent vectors",idx_list=hl_idx)
    write_prototypes(hdr_file,"gsl_vector *","DT7_t",args_impl_B,comment="//Derivatives",idx_list=hl_idx)
    write_prototypes(hdr_file,"gsl_matrix *","jac_t",args_impl_B,comment="//Jacobians",idx_list=hl_idx)
    #----explicit arguments
    write_prototypes(hdr_file,"void","T7_explicit",["gsl_vector *T","struct expl_data cr_in"])
    write_prototypes(hdr_file,"int","TmT7_explicit_t",args_expl,comment="//Tangent vectors",idx_list=hl_idx)
    write_prototypes(hdr_file,"gsl_vector *","DT7_explicit_t",args_expl,comment="//Derivatives",idx_list=hl_idx)
    write_prototypes(hdr_file,"gsl_matrix *","jac_explicit_t",args_expl,comment="//Jacobians",idx_list=hl_idx)
    #------T7
    write_prototypes(hdr_file,"double","get_T7_",args_expl,comment="//ll T7 get",idx_list=ll_idx)
    #------Derivatives
    hdr_file.write("//Elements of T7 derivatives\n")
    el=("1","2","3","4","5","6")
    for i in range(1,8):
        write_prototypes(hdr_file,"double","get_DT7_t"+str(i)+"_",args_expl,idx_list=el)  
    #------jacobians
    jac_el=[]
    for i in range(1,7):
        for j in range(1,7):
            jac_el.append(str(i)+str(j))
    hdr_file.write("//Elements of T7 derivatives\n")
    for i in range(1,8):
        write_prototypes(hdr_file,"double","get_jac_t"+str(i)+"_",args_expl,idx_list=jac_el)  
    hdr_file.write(FTR)

## Source file for high level functions

In [156]:
#---Functions with implicit arguments
def write_func_implicit(fout,ret_type,func_name,idx,func_args):
    defn=ret_type+" "+func_name+str(idx)+" ("+func_args[0]
    for a in func_args[1:]:
        defn+=","+a
    defn+=")\n{"
    defn+="""
    int i;
    double r,theta,alpha,d;
    struct expl_data cr_data;
    for(i=0;i<7;i++)
    {
        r=gsl_vector_get(bb.r,i);
        theta=gsl_vector_get(bb.dihed_angles,i);
        alpha=gsl_vector_get(bb.bend_angles, i);
   
        cr_data.r[i] = r;
        cr_data.c[i] =cos(theta);
        cr_data.s[i] =sin(theta);
        cr_data.ca[i]=cos(alpha);
        cr_data.sa[i]=sin(alpha);
    }
    for(i=0;i<7;i++)
    {
        d=gsl_vector_get(bb.d,i);
        cr_data.d[i]=d;
    }"""
    if(ret_type=="void"):
        defn+="""
    %s;\n}\n"""%(func_name+"_explicit"+str(idx)+"(T,cr_data)")
    else:
        defn+="""
    return %s;\n}\n"""%(func_name+"_explicit"+str(idx)+"(cr_data)");
    fout.write(defn)
    
#---vectors and matrices with explicit arguments
def write_expl_T7(fout):
    fn="""
void T7_explicit (gsl_vector *T, struct expl_data cr_in)
{
    gsl_vector_set(T,0,get_T7_12(cr_in));
    gsl_vector_set(T,1,get_T7_13(cr_in));
    gsl_vector_set(T,2,get_T7_23(cr_in));
    gsl_vector_set(T,3,get_T7_14(cr_in));
    gsl_vector_set(T,4,get_T7_24(cr_in));
    gsl_vector_set(T,5,get_T7_34(cr_in));
}
    """
    fout.write(fn)
    
def write_expl_vects(fout,fname,args,idx):
    defn="gsl_vector * "+fname+"_explicit_t"+str(idx)+" ("+args+")\n"
    defn+="{\n"
    defn+="\tgsl_vector *t=gsl_vector_alloc(6);\n"
    for i in range(6):
        defn+="\tgsl_vector_set(t,"+str(i)+","+"get_"+fname+"_t"+str(idx)+"_"+str(i+1)+"(cr_in));\n"
    defn+="\t return t;\n}\n"
    fout.write(defn)
    
def write_expl_matr(fout,fname,args,idx):
    defn="gsl_matrix * "+fname+"_explicit_t"+str(idx)+" ("+args+")\n"
    defn+="{\n"
    defn+="\tgsl_matrix *m=gsl_matrix_alloc(6,6);\n"
    for i in range(6):
        defn+="\t//-- \n"
        for j in range(6):
            defn+="\tgsl_matrix_set(m,"+str(i)+","+str(j)+","+"get_"+fname+"_t"+str(idx)+"_"+str(i+1)+str(j+1)+"(cr_in));\n"
    defn+="\treturn m;\n}\n"
    fout.write(defn)

#TmT7 tangent space
def write_TmT7(fout,I):
    s=""""""
    for j in range(I-1):
        s+="""
        gsl_vector_set(T,%d,gsl_vector_get(T,%d));"""%(j,j+1)
    s+="""
        gsl_vector_set(T,%d,-1);"""%(I-1)
    str_TmT7="int TmT7_t"+str(I)+"("+args_impl_A[0]+","+args_impl_A[1]+")\n{"
    str_TmT7+="""
    int i,error, sign;
    char err_string[1024];
    double r,theta,alpha,d;
    double x;
    struct expl_data cr_data;
    gsl_matrix *M;
    gsl_vector *V;
    gsl_permutation * p = gsl_permutation_alloc (6);
    for(i=0;i<7;i++)
    {
        r=gsl_vector_get(bb.r,i);
        theta=gsl_vector_get(bb.dihed_angles,i);
        alpha=gsl_vector_get(bb.bend_angles, i);
   
        cr_data.r[i] = r;
        cr_data.c[i] =cos(theta);
        cr_data.s[i] =sin(theta);
        cr_data.ca[i]=cos(alpha);
        cr_data.sa[i]=sin(alpha);
    }
    for(i=0;i<7;i++)
    {
        d=gsl_vector_get(bb.d,i);
        cr_data.d[i]=d;
    }"""
    str_TmT7+="""
    M=jac_explicit_t%d( cr_data );"""%(I)
    str_TmT7+="""
    V=DT7_explicit_t%d( cr_data );"""%(I)
    str_TmT7+="""
    error=gsl_linalg_LU_decomp(M,p,&sign);
    if(error !=GSL_SUCCESS) {
        gsl_vector_free(V);
        gsl_matrix_free(M);
        sprintf(err_string,"GSL failing with error %d\\n",error);
        failed(err_string);
    }
    x= gsl_linalg_LU_det(M,sign);
    if (fabs(x) < EPSIL_DET )
    {
        gsl_vector_free(V);
        gsl_matrix_free(M);
        gsl_permutation_free(p);
        return -1;
    }"""
    str_TmT7+="""else {
        gsl_vector_view  T_ = gsl_vector_subvector (T, 1, 6);
        error=gsl_linalg_LU_solve(M,p,V,&T_.vector);
        %s
        gsl_vector_free(V);
        gsl_matrix_free(M);
        gsl_vector_scale(T,-1.0);
        gsl_permutation_free(p);
        return error;
    }
}\n"""%(s)
    fout.write(str_TmT7)

In [157]:
with open(src_name,'w') as src_file:
    src_file.write(Timestamp+"\n")
    src_file.write("#include \""+header_name+"\"\n\n")
    src_file.write("#include <math.h>\n")
    func1="""
    void alloc_cr_input_data(cr_input_data *bb){
    bb->dihed_angles = gsl_vector_alloc(7);
    bb->bend_angles  = gsl_vector_alloc(7);
    bb->r            = gsl_vector_alloc(7);
    bb->d            = gsl_vector_alloc(7);
}

void free_cr_input_data(cr_input_data *bb){
    gsl_vector_free( bb->dihed_angles );
    gsl_vector_free( bb->bend_angles  );
    gsl_vector_free( bb->d            );
    gsl_vector_free( bb->r            );
}

void memcpy_cr_input_data(cr_input_data *bb_a, cr_input_data *bb_b){
    gsl_vector_memcpy ( bb_a->dihed_angles, bb_b->dihed_angles);
    gsl_vector_memcpy ( bb_a->bend_angles , bb_b->bend_angles );
    gsl_vector_memcpy ( bb_a->r           , bb_b->r           );
    gsl_vector_memcpy ( bb_a->d           , bb_b->d           );
}
"""
    src_file.write(func1)
    write_func_implicit(src_file,"void","T7","",args_impl_A)
    for I in range(1,8):
        write_TmT7(src_file,I)
    for I in range(1,8):
        write_func_implicit(src_file,"gsl_vector *","DT7","_t"+str(I),args_impl_A[1:])
    for I in range(1,8):
        write_func_implicit(src_file,"gsl_matrix *","jac","_t"+str(I),args_impl_A[1:])
#explicit arguments
    write_expl_T7(src_file)
    for I in range(1,8):
        write_expl_vects(src_file,"DT7","struct expl_data cr_in",I)
    for I in range(1,8):
        write_expl_matr(src_file,"jac","struct expl_data cr_in",I)
        
    # write_func_expl_hl(src_file,"")

## Source files for get functions

This section produces three source files: one for T7 equations, one for their derivatives and the third for their jacobians.

In [158]:
def write_expl_function(fout,ret_type, func_prefix, func_name, func_args=[]):
    with open(func_name+".c", 'r') as f_in:
        equation=f_in.readline()
    ##
    defn=ret_type+" "+func_prefix+func_name+" ("+func_args[0]
    for a in func_args[1:]:
        defn+=","+a
    defn+=")\n{"
    fout.write(defn+"\n")
    for i in range(1,8):
        fout.write("\tdouble c"+ str(i)+" = cr_in.c ["+str(i-1)+"];\n")
        fout.write("\tdouble s"+ str(i)+" = cr_in.s ["+str(i-1)+"];\n")
        fout.write("\tdouble ca"+ str(i)+"= cr_in.ca["+str(i-1)+"];\n")
        fout.write("\tdouble sa"+ str(i)+"= cr_in.sa["+str(i-1)+"];\n")
        fout.write("\tdouble r"+ str(i)+" = cr_in.r ["+str(i-1)+"];\n")
        fout.write("\tdouble d"+ str(i)+" = cr_in.d ["+str(i-1)+"];\n")
    #for i in range(1,8):
        
    fout.write("\treturn "+equation+";\n}\n")

In [159]:
#Write Equations
ll_idx=("12","13","23","14","24","34","33")
with open(eqn_fname,'w') as eqn_file:
    eqn_file.write(Timestamp+"\n")
    eqn_file.write("#include \""+header_name+"\"\n\n")
    for idx in ll_idx:
        fname="T7_"+idx
        write_expl_function(eqn_file,"double", "get_",fname,func_args=args_expl)
#Write Derivatives
with open(der_fname,'w') as der_file:
    der_file.write(Timestamp+"\n")
    der_file.write("#include \""+header_name+"\"\n\n")
    for i in range(1,8):
        for j in range(1,7):
            fname="DT7_t"+str(i)+"_"+str(j)
            write_expl_function(der_file,"double", "get_",fname,func_args=args_expl)
#Write Jacobians
with open(jac_fname,'w') as jac_file:
    jac_file.write(Timestamp+"\n")
    jac_file.write("#include \""+header_name+"\"\n\n")
    for i in range(1,8):
        for j in range(1,7):
            for k in range(1,7):
                fname="jac_t"+str(i)+"_"+str(j)+str(k)
                write_expl_function(jac_file,"double", "get_",fname,func_args=args_expl)

